# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import psycopg2.extras
import pandas as pd
from sql_queries import *
import  create_tables

In [2]:
create_tables.main()

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_data_path = './data/song_data'
song_files = get_files(song_data_path)

In [6]:
# Read all song files into one dataframe
df = pd.concat((pd.read_json(file, lines=True) for file in song_files))
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7SMBG1187B9B9066,NaN,NaN,,Los Manolos,SOBCOSW12A8C13D398,Rumba De Barcelona,218.38322,0
0,1,ARBGXIG122988F409D,37.77916,-122.42005,California - SF,Steel Rain,SOOJPRH12A8C141995,Loaded Like A Gun,173.19138,0
0,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0
0,1,ARP6N5A1187B99D1A3,NaN,NaN,"Hamtramck, MI",Mitch Ryder,SOXILUQ12A58A7C72A,Jenny Take a Ride,207.43791,2004
0,1,AR36F9J1187FB406F1,56.27609,9.51695,Denmark,Bombay Rockers,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),230.71302,0


In [7]:
# Drop rows where the song_id is identical - Not really necessary
df = df.drop_duplicates(subset='song_id', keep='last')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = list(df[columns].values[0])
song_data

['SOBCOSW12A8C13D398',
 'Rumba De Barcelona',
 'AR7SMBG1187B9B9066',
 0,
 218.38322]

In [9]:
title_list = list(df['title'])

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
df_songs_data = df[columns]

In [11]:
df_songs_data.head()

,song_id,title,artist_id,year,duration
0,SOBCOSW12A8C13D398,Rumba De Barcelona,AR7SMBG1187B9B9066,0,218.38322
0,SOOJPRH12A8C141995,Loaded Like A Gun,ARBGXIG122988F409D,0,173.19138
0,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0,407.37914
0,SOXILUQ12A58A7C72A,Jenny Take a Ride,ARP6N5A1187B99D1A3,2004,207.43791
0,SOBKWDJ12A8C13B2F3,Wild Rose (Back 2 Basics Mix),AR36F9J1187FB406F1,0,230.71302


In [12]:
# temp_dir = './temp_copy_to_sql/'
# os.makedirs(temp_dir, exist_ok=True)
# temp_file = temp_dir+'temp_copy_to_sql.csv'
# df_songs_data.to_csv(temp_file, header=False, index=False, sep='|', quotechar="'")
# with open(temp_file, 'r') as f_in:
#     cur.copy_from(f_in, 'songs', sep='|')
#     conn.commit()

In [13]:
def insert_df_into_table(cur, table, df):
    """Inserting values in a dataframe into the database.

    Inserts values from a dataframe into a table. 
    The dataframe can only contain the columns to be inserted.

    Args:
        cur (psycopg2.extensions.cursor): Cursor to the postgres database.
        table (str): The name of the table to insert the data into.
        df (pandas.Dataframe): Dataframe with the data to insert into the table.
    """

    temp_dir = './temp_copy_to_sql/'
    os.makedirs(temp_dir, exist_ok=True)
    temp_file = temp_dir+'temp_copy_to_sql.csv'
    df.to_csv(temp_file, header=False, index=False, sep='|', quotechar="'", na_rep='NaN')
    with open(temp_file, 'r') as f_in:
        cur.copy_from(f_in, table, sep='|')
        conn.commit()

    os.remove(temp_file)
    if not os.listdir(temp_dir):
        os.rmdir(temp_dir)

In [14]:
try:
    insert_df_into_table(cur, 'songs', df[columns])
    conn.commit()
except psycopg2.Error as e:
    print('Inserting data did not complete without errors.')
    print(e)
    print('Connection rollback.')
    conn.rollback()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [16]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

df_artist = df[artist_columns]
df_artist = df_artist.drop_duplicates(subset='artist_id', keep='last')

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
df_artist['artist_id'].nunique()

69

In [19]:
try:
    insert_df_into_table(cur, 'artists', df_artist)
    conn.commit()
except psycopg2.Error as e:
    print('Inserting data did not complete without errors.')
    print(e)
    print('Connection rollback.')
    conn.rollback()

Inserting data did not complete without errors.
duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(AR7SMBG1187B9B9066) already exists.
CONTEXT:  COPY artists, line 1

Connection rollback.


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [20]:
log_files_path = 'data/log_data'
log_files = get_files(log_files_path)

In [21]:
df = pd.concat((pd.read_json(file, lines=True) for file in log_files), axis=0)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
df = df[df['page'] == 'NextSong']

In [23]:
t = pd.to_datetime(df['ts'], unit='ms', origin='unix')
t.head()

6    2018-11-21 00:57:58.796
7    2018-11-21 00:58:41.796
8    2018-11-21 01:02:05.796
9    2018-11-21 01:03:13.796
11   2018-11-21 01:05:29.796
Name: ts, dtype: datetime64[ns]

In [24]:
time_data = list(zip(t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [25]:
df_time = pd.DataFrame(time_data, columns=column_labels)
df_time = df_time.drop_duplicates()
df_time.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-21 00:57:58.796,0,21,47,11,2018,2
1,2018-11-21 00:58:41.796,0,21,47,11,2018,2
2,2018-11-21 01:02:05.796,1,21,47,11,2018,2
3,2018-11-21 01:03:13.796,1,21,47,11,2018,2
4,2018-11-21 01:05:29.796,1,21,47,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [27]:
try:
    insert_df_into_table(cur, 'time', df_time)
    conn.commit()
except psycopg2.Error as e:
    print('Inserting data did not complete without errors.')
    print(e)
    print('Connection rollback.')
    conn.rollback()

Inserting data did not complete without errors.
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-21 00:57:58.796) already exists.
CONTEXT:  COPY time, line 1

Connection rollback.


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [28]:
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
df_user = df[user_columns]

In [29]:
# check data quality
df = df.dropna(axis=0)
df = df[df['userId'] != '']
df_user['userId'] = df_user['userId'].astype(int)
df_user = df_user.drop_duplicates(subset='userId', keep='last')


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
try:
    insert_df_into_table(cur, 'users', df_user)
    conn.commit()
except psycopg2.Error as e:
    print('Inserting data did not complete without errors.')
    print(e)
    print('Connection rollback.')
    conn.rollback()

Inserting data did not complete without errors.
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(65) already exists.
CONTEXT:  COPY users, line 1

Connection rollback.


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [33]:
df['songplay_id'] = df['sessionId'].astype(str) + '-' + df['ts'].astype(str)
df['start_time'] = pd.to_datetime(df.ts, unit='ms', origin='unix')
songplay_columns = ['songplay_id', 'start_time', 'userId', 'level', 'song', 
                    'artist', 'sessionId', 'location', 'userAgent', 'length']
df_songplay = df[songplay_columns].reset_index(drop=True)
df_songplay
# # insert into temporary table
cur.execute(temp_log_data_create)
conn.commit()
try:
    insert_df_into_table(cur, 'temp_log_data', df_songplay)
    conn.commit()
except psycopg2.Error as e:
    print('Inserting data did not complete without errors.')
    print(e)
    print('Connection rollback.')
    conn.rollback()

In [34]:
# Join with the data in songs and artists, and insert into songplays
cur.execute(join_log_data_songs_artists)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [35]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.